In [1]:
# 抓取个股2015年k线数据
from opentrader.core.crawler import *
from opentrader.agents.xueqiu.api import *
from opentrader.agents.ths.api import *
c = CNCrawler()
stock_list = c.get_stock_list()


Querying 54 pages ......................................................

In [2]:
from datetime import datetime, timedelta
from pymongo.errors import *
from opentrader.common.db import db_ot

def fetch_year_kday(year):
    print('---------------year: '+year+'--------------------')
    ths = THSAPI()
    for i,stock in enumerate(stock_list):
        if i%10 == 0:
            print('.', end='')
        #print('Querying stock %d, %s...' % (i, stock['symbol']), end=' | ')
        #result = db_ot.ths_kday.find({'symbol':stock['symbol'], 'date':datetime.strptime('20101217', '%Y%m%d')}).count()
        #print(result)
        #if result > 0:
        #    continue
        try:
            data = ths._get_market_data(stock['symbol'], query=year+'.js', flag='01')
        except InvalidSymbol:
            continue
        if len(data) == 0:
            continue
        insert_count = 0
        dup_count = 0
        error_count = 0
        for line in data.split(';'):
            line = line.split(',')
            dayk = {
                'symbol':stock['symbol'],
                'date':datetime.strptime(line[0], '%Y%m%d'),
                'open':line[1],
                'high':line[2],
                'low':line[3],
                'close':line[4],
                'volume':line[5],
                'amount':line[6],
                'turnrate':line[7],
            }
            try:
                result = db_ot.ths_kday.insert_one(dayk)
                if result:
                    insert_count += 1
                else:
                    error_count += 1
            except DuplicateKeyError as e:
                dup_count += 1
        #print("Stock %s OK, inserted: %d, duplicate: %d, error: %d" % (stock['symbol'], insert_count, dup_count, error_count))

for year in ('2007','2008','2009','2010','2011','2012','2013','2014','2015','2016'):
    fetch_year_kday(year)

---------------year: 2007--------------------
.........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................---------------year: 2008--------------------
...................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [6]:
from opentrader.agents.ths.api import *
ths = THSAPI()
ths._get_market_data('SZ002345', query='today.js', flag='01')

'idge_v2_line_hs_002345_01_today({"hs_002345":{"1":"20160112","7":"11.41","8":"11.67","9":"11.01","11":"11.44","13":15214641,"19":"172828280.00","1968584":"1.828","open":1,"dt":"1508","name":"\\u6f6e\\u5b8f\\u57fa"'